In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
df=pd.read_csv('data.csv')
df.head()
stops=stopwords.words('english')

In [3]:
real=' '.join(df[df['Label']==1]['Headline'])
fake=' '.join(df[df['Label']==0]['Headline'])



In [4]:
y_real=np.array([1 for i in range(1872)])
y_fake=np.array([0 for i in range(2137)])

In [5]:
from random import shuffle

real_list=list(df[df['Label']==1]['Headline'].values)

fake_list=list(df[df['Label']==0]['Headline'].values)


total_words=real_list+fake_list

real_data=list(zip(real_list,y_real))
fake_data=list(zip(fake_list,y_fake))

data=real_data+fake_data

shuffle(data)

In [6]:
x_train=[i[0] for i in data[:3009]]
y_train=np.array([i[1] for i in data[:3009]])
x_test=[i[0] for i in data[3009:]]
y_test=np.array([i[1] for i in data[3009:]])







In [7]:
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(total_words)

In [8]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)



In [9]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=69+2*44,
                            padding='pre', truncating='pre')
x_test_pad = pad_sequences(x_test_tokens, maxlen=69+2*44,
                           padding='pre', truncating='pre')



In [10]:
model = Sequential()
model.add(Embedding(input_dim=7879,
                    output_dim=10,
                    input_length=69+2*44,
                    name='layer_embedding'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=10, batch_size=64)

Train on 2858 samples, validate on 151 samples
Epoch 1/10
2858/2858 [==============================] - 40s - loss: 0.6928 - acc: 0.5245 - val_loss: 0.6915 - val_acc: 0.5563
Epoch 2/10
2858/2858 [==============================] - 29s - loss: 0.6868 - acc: 0.5549 - val_loss: 0.6943 - val_acc: 0.4901
Epoch 3/10
2858/2858 [==============================] - 29s - loss: 0.6729 - acc: 0.5934 - val_loss: 0.6823 - val_acc: 0.5099
Epoch 4/10
2858/2858 [==============================] - 29s - loss: 0.6042 - acc: 0.7050 - val_loss: 0.6021 - val_acc: 0.6821
Epoch 5/10
2858/2858 [==============================] - 29s - loss: 0.4259 - acc: 0.8181 - val_loss: 0.5514 - val_acc: 0.7417
Epoch 6/10
2858/2858 [==============================] - 29s - loss: 0.3181 - acc: 0.8712 - val_loss: 0.5260 - val_acc: 0.7550
Epoch 7/10
2858/2858 [==============================] - 29s - loss: 0.2338 - acc: 0.9199 - val_loss: 0.5156 - val_acc: 0.7748
Epoch 8/10
2858/2858 [==============================] - 29s - loss: 0.1

In [11]:
result = model.evaluate(x_test_pad, y_test)
result[1]

1000/1000 [==============================] - 4s     


0.81000000000000005

In [12]:
req=input('enter the news\n')
req=[req]
req_token=tokenizer.texts_to_sequences(req)



req_pad=pad_sequences(req_token, maxlen=69+2*44,
                           padding='pre', truncating='pre')

pred=model.predict([req_pad])


print('FAKE NEWS!!') if pred>0.6 else print("REAL NEWS!!")


enter the news
Narendra Modi just bought a lamborghini
FAKE NEWS!!
